In [6]:
import json
import pandas as pd
import fitz
import unicodedata
import os
from pathlib import Path

def normalize_vietnamese(text):
    return unicodedata.normalize('NFC', text)

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def load_excel(file_path):
    return pd.read_excel(file_path).to_dict('records')

def load_pdf(file_path):
    doc = fitz.open(file_path)
    pages = []
    for page_num, page in enumerate(doc):
        text = page.get_text()
        pages.append({
            'content': normalize_vietnamese(text),
            'metadata': {
                'title': os.path.basename(file_path),
                'source': 'pdf',
                'file_name': os.path.basename(file_path),
                'page': page_num + 1,
                'category': 'culture_tourism',
                'language': 'vi',
                'geo': None,
                'original_id': f'pdf_{page_num}'
            }
        })
    return pages

def main():
    base_dir = Path.cwd().parent  # Thư mục hiện tại của notebook
    input_dir = base_dir / "data"
    output_file = input_dir / "merged_docs.json"

    files = [
        'culture_vi_enriched.json',
        'CoSoDatChuan.json',
        'KhuDiemDuLich.xlsx',
        'stops_hcm.json',
        'VanHoaVaDuLichVN.pdf'
    ]

    docs = []

    for file in files:
        path = input_dir / file
        print(f"Đang xử lý: {file}")

        if file.endswith('.json'):
            data = load_json(path)
            for item in data if isinstance(data, list) else [data]:
                docs.append({
                    'content': normalize_vietnamese(str(item)),
                    'metadata': {
                        'title': file,
                        'source': 'json',
                        'file_name': file,
                        'page': None,
                        'category': 'transport' if 'stops' in file else 'culture_tourism',
                        'language': 'vi',
                        'geo': item.get('coordinates') if isinstance(item, dict) and 'coordinates' in item else None,
                        'original_id': item.get('id', str(len(docs))) if isinstance(item, dict) else str(len(docs))
                    }
                })

        elif file.endswith('.xlsx'):
            data = load_excel(path)
            for item in data:
                docs.append({
                    'content': normalize_vietnamese(str(item)),
                    'metadata': {
                        'title': file,
                        'source': 'excel',
                        'file_name': file,
                        'page': None,
                        'category': 'tourism',
                        'language': 'vi',
                        'geo': item.get('lat_lon') if 'lat_lon' in item else None,
                        'original_id': item.get('id', str(len(docs)))
                    }
                })

        elif file.endswith('.pdf'):
            docs.extend(load_pdf(path))

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(docs, f, ensure_ascii=False, indent=2)

    print(f"\nHoàn tất. Tổng số tài liệu: {len(docs)}")
    print(f"Đã lưu tại: {output_file}")

main()


Đang xử lý: culture_vi_enriched.json
Đang xử lý: CoSoDatChuan.json
Đang xử lý: KhuDiemDuLich.xlsx
Đang xử lý: stops_hcm.json
Đang xử lý: VanHoaVaDuLichVN.pdf

Hoàn tất. Tổng số tài liệu: 5494
Đã lưu tại: d:\RAG\data\merged_docs.json
